In [1]:
import cuml
import cudf
import cupy

In [2]:
#Importing Dataset
df=cudf.read_csv('ab.csv')

In [3]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   y       236 non-null    float64
 1   x       236 non-null    float64
dtypes: float64(2)
memory usage: 3.7 KB


#Fitting a Linear Regression model

In [5]:
#Seperating Input and Output Features
X=df.iloc[:,1]
Y=df.iloc[:,0]

In [8]:
#Scaling input features
from cuml.preprocessing import StandardScaler
ss=StandardScaler()
X_scaled=ss.fit_transform(cudf.DataFrame(X))

In [9]:
X_scaled=cudf.DataFrame(X_scaled)

In [10]:
#Adding a column of Ones
X_scaled.insert(loc=0,value=1.0,name='Ones')

In [11]:
#train-test_split
from cuml.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X_scaled,Y,test_size=0.30,random_state=42)

In [12]:
#finding beta's using LSE
A=X_train.values.T.dot(X_train.values)
B=cupy.linalg.inv(A)
C=B.dot(X_train.values.T)
beta=C.dot(Y_train.values)

In [13]:
beta

array([14.14973391,  4.78224172])

In [14]:
#Predicting and Testing
Y_predict=X_test.values.dot(beta)
from cuml import metrics
metrics.r2_score(Y_test,Y_predict)

0.6719866616378707

In [15]:
#adding powers of X
from cuml.preprocessing import PolynomialFeatures
pf=PolynomialFeatures(5)
X5=pf.fit_transform(cudf.DataFrame(X))

X5

In [16]:
X5

,0,1,2,3,4,5
0,1.0,24.4,595.36,1.452678e+04,3.544535e+05,8.648666e+06
1,1.0,34.8,1211.04,4.214419e+04,1.466618e+06,5.103830e+07
2,1.0,44.1,1944.81,8.576612e+04,3.782286e+06,1.667988e+08
3,1.0,45.1,2034.01,9.173385e+04,4.137197e+06,1.865876e+08
4,1.0,55.0,3025.00,1.663750e+05,9.150625e+06,5.032844e+08
...,...,...,...,...,...,...
231,1.0,850.0,722500.00,6.141250e+08,5.220062e+11,4.437053e+14
232,1.0,852.0,725904.00,6.184702e+08,5.269366e+11,4.489500e+14
233,1.0,850.0,722500.00,6.141250e+08,5.220062e+11,4.437053e+14
234,1.0,851.0,724201.00,6.162951e+08,5.244671e+11,4.463215e+14


In [17]:
#scaling features
ss1=StandardScaler()
X5_scaled=ss1.fit_transform(X5.iloc[:,1:])

In [18]:
#Inserting columns of One
X5_scaled.insert(loc=0,value=1.0,name='Ones')

In [19]:
#Train -Test Split
from cuml.model_selection import train_test_split
X_train1,X_test1,Y_train1,Y_test1=train_test_split(X5_scaled,Y,test_size=0.30,random_state=42)

In [20]:
#Finding beta's for 5th order hypothesis function
A=X_train1.values.T.dot(X_train1.values)
B=cupy.linalg.inv(A)
C=B.dot(X_train1.values.T)
beta1=C.dot(Y_train1.values)

In [21]:
beta1

array([  14.21778854,   50.92380085, -169.95617705,  274.20599143,
       -216.87424495,   67.16997504])

In [22]:
#prediciting and testing
Y_predict1=X_test1.values.dot(beta1)
from cuml import metrics
metrics.r2_score(Y_test1,Y_predict1)

0.9979841651699113

In [23]:
#Adding powers of input feature (0 to 50)
from cuml.preprocessing import PolynomialFeatures
pf=PolynomialFeatures(50)
X50=pf.fit_transform(cudf.DataFrame(X))

In [24]:
X50.shape

(236, 51)

In [25]:
#Scaling features
ss1=StandardScaler()
X50_scaled=ss1.fit_transform(X50.iloc[:,1:])

In [26]:
#Inserting a column
X50_scaled.insert(loc=0,value=1.0,name='Ones')

In [27]:
#Train Test Split
from cuml.model_selection import train_test_split
X_train2,X_test2,Y_train2,Y_test2=train_test_split(X50_scaled,Y,test_size=0.30,random_state=42)

In [28]:
#Finding beta's for 50 order hypothesis function
A=X_train2.values.T.dot(X_train2.values)
B=cupy.linalg.inv(A)
C=B.dot(X_train2.values.T)
beta2=C.dot(Y_train2.values)

In [29]:
#Predicitng and testing
Y_predict2=X_test2.values.dot(beta2)
from cuml import metrics
metrics.r2_score(Y_test2,Y_predict2)

-2.106049393274842

In [30]:
#finding beta's with ridge regression
lamda=0.001
A=X_train2.values.T.dot(X_train2.values)
I=cupy.identity(A.shape[0])
A1=cupy.add(A,lamda*I)
B=cupy.linalg.inv(A1)
C=B.dot(X_train2.values.T)
beta3=C.dot(Y_train2.values)

In [31]:
#Predicting and Testing the ridge regression model
Y_predict3=X_test2.values.dot(beta3)
from cuml import metrics
metrics.r2_score(Y_test2,Y_predict3)

0.9963337036942519

In [34]:
#tuning the value of lamda (regularization parameter)
lamda=[1e-7,1e-6,1e-5,1e-4,1e-3,1e-2,1e-1,1,10,100,1000]
R2_score=[]
for i in range(len(lamda)):
    A=X_train2.values.T.dot(X_train2.values)
    I=cupy.identity(A.shape[0])
    A1=cupy.add(A,lamda[i]*I)
    B=cupy.linalg.inv(A1)
    C=B.dot(X_train2.values.T)
    beta3=C.dot(Y_train2.values)
    Y_predict3=X_test2.values.dot(beta3)
    R2_score.append(metrics.r2_score(Y_test2,Y_predict3))

In [35]:
R2_score


[0.9985898493350581,
 0.998186592939224,
 0.9980202066612831,
 0.9980369922130834,
 0.9963337036942519,
 0.9881353456026424,
 0.9629476699168672,
 0.9079916332138837,
 0.7605953829804118,
 -0.4148540643998686,
 -4.620807822568909]